In [30]:
import pandas as pd
from datetime import datetime

# Load raw data
df_raw = pd.read_csv("../data/simulated/claims_raw_simulated.csv")

df = df_raw.copy()

In [31]:

# -----------------------
# Standardize column names
# -----------------------
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

In [32]:

# -----------------------
# Clean salary (robust to missing values)
#    - keep proper missing values (avoid turning NaN into "nan")
# -----------------------
salary_s = (
    df["annual_salary_raw"]
    .astype("string")  # keeps <NA> instead of "nan"
    .str.replace("€", "", regex=False)
    .str.replace("EUR", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.replace(" ", "", regex=False)
)
df["annual_salary"] = pd.to_numeric(salary_s, errors="coerce")

In [33]:
# -----------------------
# numeric fields
# -----------------------

df["claim_duration_days"] = pd.to_numeric(df["claim_duration_days_raw"], errors="coerce")
df["daily_benefit"] = pd.to_numeric(df["daily_benefit_raw"], errors="coerce")

In [34]:
# -----------------------
# Parse dates (France-friendly) + numeric fields
# --- helper: robust date parsing for mixed formats ---
# -----------------------

def parse_date_mixed_strict(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip()

    # Normalize separators: "/" and "." -> "-"
    s_norm = (s
              .str.replace("/", "-", regex=False)
              .str.replace(".", "-", regex=False)
             )

    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    # 1) Try YYYY-MM-DD explicitly (fast + unambiguous)
    mask = out.isna() & s_norm.notna()
    out.loc[mask] = pd.to_datetime(s_norm.loc[mask], format="%Y-%m-%d", errors="coerce")

    # 2) Try DD-MM-YYYY explicitly (French style)
    mask = out.isna() & s_norm.notna()
    out.loc[mask] = pd.to_datetime(s_norm.loc[mask], format="%d-%m-%Y", errors="coerce")

    return out

# Apply
df["date_of_birth"] = parse_date_mixed_strict(df["date_of_birth"])
df["claim_start_date"] = parse_date_mixed_strict(df["claim_start_date_raw"])

In [35]:
# -----------------------
# Calculate age at claim date
# -----------------------
df["age_at_claim"] = (df["claim_start_date"] - df["date_of_birth"]).dt.days / 365.25

In [36]:
# -----------------------
# Deduplicate intelligently: keep the most complete record
# -----------------------

dup_keys = ["insured_id", "claim_start_date", "claim_duration_days_raw", "daily_benefit_raw"]

# score: how many critical fields are present (higher = better)
df["completeness_score"] = (
    df["date_of_birth"].notna().astype(int)
    + df["annual_salary"].notna().astype(int)
    + df["claim_start_date"].notna().astype(int)
    + df["claim_duration_days"].notna().astype(int)
    + df["daily_benefit"].notna().astype(int)
)

# sort so that the "best" row comes first within each duplicate group
df = df.sort_values(by=dup_keys + ["completeness_score"], ascending=[True, True, True, True, False])

# now drop duplicates keeping the first (= best)
df = df.drop_duplicates(subset=dup_keys, keep="first").drop(columns=["completeness_score"])

,insured_id,date_of_birth,annual_salary_raw,claim_start_date_raw,claim_duration_days_raw,daily_benefit_raw,annual_salary,claim_duration_days,daily_benefit,claim_start_date,age_at_claim
0,A001,1978-04-12,45 000,2023/03/12,30.0,110,45000,30.0,110,2023-03-12,44.914442
2,A002,1984-11-03,38000€,05-02-2023,15.0,90,38000,15.0,90,2023-02-05,38.255989
3,A003,1940-01-15,25000,2022/14/01,200.0,70,25000,200.0,70,NaT,NaN
4,A004,1992-07-21,"52,000",2023-06-01,10.0,130,52000,10.0,130,2023-06-01,30.861054
5,A005,1988-03-05,NaN,2023-08-15,25.0,95,<NA>,25.0,95,2023-08-15,35.444216
6,A006,1970-12-30,41000 EUR,2023.09.10,40.0,105,41000,40.0,105,2023-09-10,52.695414
7,A007,2005-01-01,22000,2023-04-20,5.0,60,22000,5.0,60,2023-04-20,18.297057
8,A008,1965-05-18,60000,invalid_date,20.0,150,60000,20.0,150,NaT,NaN
9,A009,1980-09-09,47000,2023-11-02,NaN,120,47000,NaN,120,2023-11-02,43.145791


In [38]:
# -----------------------
# Filters / validations (V1)
# -----------------------
df_clean = df[
    df["claim_start_date"].notna()
    & df["date_of_birth"].notna()
    & df["annual_salary"].notna()
    & df["claim_duration_days"].notna()
    & df["daily_benefit"].notna()
    & (df["age_at_claim"].between(18, 70))
    & (df["claim_duration_days"].between(1, 365))
].copy()

,insured_id,date_of_birth,annual_salary_raw,claim_start_date_raw,claim_duration_days_raw,daily_benefit_raw,annual_salary,claim_duration_days,daily_benefit,claim_start_date,age_at_claim
0,A001,1978-04-12,45 000,2023/03/12,30.0,110,45000,30.0,110,2023-03-12,44.914442
2,A002,1984-11-03,38000€,05-02-2023,15.0,90,38000,15.0,90,2023-02-05,38.255989
4,A004,1992-07-21,"52,000",2023-06-01,10.0,130,52000,10.0,130,2023-06-01,30.861054
6,A006,1970-12-30,41000 EUR,2023.09.10,40.0,105,41000,40.0,105,2023-09-10,52.695414
7,A007,2005-01-01,22000,2023-04-20,5.0,60,22000,5.0,60,2023-04-20,18.297057


In [39]:
# Optional: keep only columns we care about (nice for downstream work)
df_clean = df_clean[
    [
        "insured_id",
        "date_of_birth",
        "claim_start_date",
        "age_at_claim",
        "annual_salary",
        "claim_duration_days",
        "daily_benefit",
    ]
]
df_clean

,insured_id,date_of_birth,claim_start_date,age_at_claim,annual_salary,claim_duration_days,daily_benefit
0,A001,1978-04-12,2023-03-12,44.914442,45000,30.0,110
2,A002,1984-11-03,2023-02-05,38.255989,38000,15.0,90
4,A004,1992-07-21,2023-06-01,30.861054,52000,10.0,130
6,A006,1970-12-30,2023-09-10,52.695414,41000,40.0,105
7,A007,2005-01-01,2023-04-20,18.297057,22000,5.0,60


In [40]:
# -----------------------
# Save cleaned data
# -----------------------
df_clean.to_csv("../data/processed/claims_cleaned.csv", index=False)

In [41]:
# Quick check
print("Rows raw:", len(df_raw))
print("Rows cleaned:", len(df_clean))
df_clean.head(10)

Rows raw: 10
Rows cleaned: 5


,insured_id,date_of_birth,claim_start_date,age_at_claim,annual_salary,claim_duration_days,daily_benefit
0,A001,1978-04-12,2023-03-12,44.914442,45000,30.0,110
2,A002,1984-11-03,2023-02-05,38.255989,38000,15.0,90
4,A004,1992-07-21,2023-06-01,30.861054,52000,10.0,130
6,A006,1970-12-30,2023-09-10,52.695414,41000,40.0,105
7,A007,2005-01-01,2023-04-20,18.297057,22000,5.0,60
